In [ ]:
!pip install transformers tensorflow datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

import tensorflow_datasets as tfds
from datasets import Dataset

In [ ]:
# Load the IMDB dataset
(ds_train_tf, ds_test_tf), ds_info = tfds.load(
    'imdb_reviews',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.M90RO4_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.M90RO4_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.M90RO4_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# Ensure this line is present and runs before using the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convert tf.data.Dataset to list of texts and labels
def convert_to_lists(dataset):
    texts, labels = [], []
    for text, label in tfds.as_numpy(dataset):
        texts.append(text.decode('utf-8'))
        labels.append(label)
    return texts, labels

train_texts, train_labels = convert_to_lists(ds_train_tf)
test_texts, test_labels = convert_to_lists(ds_test_tf)

# Build HuggingFace datasets
ds_train = Dataset.from_dict({
    'text': train_texts,
    'label': train_labels,
})

ds_test = Dataset.from_dict({
    'text': test_texts,
    'label': test_labels,
})

# Define preprocess function
def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Map datasets
ds_train_encoded = ds_train.map(preprocess_function, batched=True, remove_columns=['text'])
ds_test_encoded = ds_test.map(preprocess_function, batched=True, remove_columns=['text'])

# Verify and reformat datasets suitably
ds_train_encoded.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])
ds_test_encoded.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
# Fine-tune BERT
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Use the Adam optimizer directly
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model.summary())

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


# WARNING!
- This Model Require an A100 GPU with 33GB of VRAM to Train!

In [ ]:
# Train the model
model.fit(
    ds_train_encoded.to_tf_dataset(
        columns=['input_ids', 'attention_mask'],
        label_cols=['label'],  # Correct label column name
        batch_size=32,
        shuffle=True
    ),
    epochs=3
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


Epoch 1/3
782/782 [==============================] - 369s 420ms/step - loss: 0.2458 - accuracy: 0.8998
Epoch 2/3
782/782 [==============================] - 328s 419ms/step - loss: 0.1232 - accuracy: 0.9565
Epoch 3/3
782/782 [==============================] - 328s 419ms/step - loss: 0.0699 - accuracy: 0.9764


In [ ]:
# Evaluate the model
results = model.evaluate(
    ds_test_encoded.to_tf_dataset(
        columns=['input_ids', 'attention_mask'],
        label_cols=['label'],  # Specify the correct label column
        batch_size=8
    ),
)

print(f"Test accuracy: {results[1]}")

3125/3125 [==============================] - 154s 48ms/step - loss: 0.2106 - accuracy: 0.9360
Test accuracy: 0.9360399842262268


In [ ]:
# Save the model's weights
weights_file_path = 'model_weights.h5'  # Choose a suitable path and filename
model.save_weights(weights_file_path)

print(f"Model weights saved to {weights_file_path}")

Model weights saved to model_weights.h5


## Inferencing the Model

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow_datasets as tfds
from datasets import Dataset

In [ ]:
# Create a model instance with the same architecture
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Load the previously saved weights
weights_file_path = 'model_weights.h5'  # Replace with the actual path
model.load_weights(weights_file_path)

print(f"Weights loaded successfully from {weights_file_path}")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Weights loaded successfully from model_weights.h5


In [ ]:
sample_texts = [
    "The movie was fantastic!",
    "I didn't enjoy the film as much as I thought I would.",
    "The plot was predictable, but the acting was top-notch.",
    "I've never seen anything quite like it. Truly unique experience.",
    "I would recommend this book to everyone.",
    "The weather today is quite pleasant compared to yesterday.",
    "He made a compelling argument during the lecture.",
    "The new restaurant in town exceeded my expectations.",
    "I've heard a lot about this place; finally got to visit it!",
    "Her performance on stage was breathtaking.",
    "I've never been more disappointed by a season finale.",
    "Can you believe how amazing that concert was last night?",
    "His latest novel is a gripping tale of adventure.",
    "Have you ever tried skydiving? It's exhilarating!",
    "The cafe on Main Street has the best coffee in town.",
    "Their customer service deserves high praise.",
    "It's one of the worst movies I've ever seen.",
    "I love spending my weekends hiking in the mountains.",
    "The cheese platter was the highlight of the evening.",
    "Learning new languages has always been a passion of mine.",
    "Did you catch the game last night? What a thriller!",
    "The app's user interface is very intuitive and user-friendly.",
    "I've heard mixed reviews about the latest gadget launch.",
    "The museum exhibition was both informative and entertaining.",
    "I wish I had started investing in stocks a decade ago.",
    "Gardening has really helped me manage stress better.",
    "The live performance far exceeded my expectations.",
    "Staying organized is the key to balancing work and life.",
    "I didn't expect the recipe to turn out so well on the first try!",
    "His mentorship profoundly impacted my career path.",
    "What a delightful surprise to receive your letter today!",
    "The park is a great place for a picnic this weekend.",
    "I'm fascinated by technology's rapid advancement in recent years.",
    "The trip to Japan was a cultural enrichment beyond measure.",
    "Cooking dinner together has become our weekend tradition.",
    "She's one of the most inspiring leaders I've ever worked with.",
    "How do you stay motivated during tough times?",
    "The mountain trek was challenging, but we made it to the top!",
    "Our team has notable achievements this quarter.",
    "I can't wait for the family reunion this summer!",
    "Yoga has immensely benefited my physical and mental health.",
    "He's thinking about renovating his home office.",
    "The car ride to the countryside was scenic and peaceful.",
    "Why not make homemade pizza for dinner tonight?",
    "Attending the conference was a worthwhile experience.",
    "Her artwork is featured in the city's annual gallery.",
    "The solution to climate change requires collective action.",
    "I finally finished reading that lengthy, complex novel.",
    "Taking daily breaks has improved my productivity at work.",
    "Tonight's full moon is mesmerizing and serene.",
    "The recent documentary opened my eyes to environmental issues.",
    "She's been training for the marathon for months now.",
    "Do you think virtual reality is the future of gaming?",
    "The charity event was a great success this year.",
    "I've learned so much from online courses during the lockdown.",
    "The bustling market was full of lively vendors and vibrant goods.",
    "What an epic showdown in the season's opening episode!",
    "The basement renovation turned into quite the DIY project.",
    "Camping over the weekend is something I look forward to every month.",
    "Her dedication to volunteering is an inspiration to all.",
    "Discovering new music genres has broadened my taste.",
    "Do you follow any podcasts? They're a great source of information.",
    "We spent the day exploring hidden gems in the city.",
    "It's amazing how technology evolves faster every year.",
    "The bakery's chocolate croissants are a morning indulgence.",
    "Consider adopting a pet from the local shelter!",
    "The team's hard work culminated in an industry award.",
    "He enjoys tinkering with vintage car engines in his spare time.",
    "Starting my day with meditation has been transformative.",
    "What strategies do you use for effective time management?",
    "The festival showcased a diverse range of international cuisines.",
    "It's thrilling to witness the evolution of artificial intelligence.",
    "The early morning mist created an enchanting atmosphere.",
    "Preparing for interviews is a crucial step in the job search.",
    "The cabin in the woods was a perfect retreat from city life.",
    "The science fiction series captivated audiences worldwide.",
    "Setting realistic goals is vital for steady progress.",
    "It's crucial to support local businesses during tough times.",
    "The garden is thriving with colorful blooms this spring.",
    "Ocean conservation deserves more attention and resources.",
    "The sunset over the ocean was breathtakingly beautiful.",
    "I aspire to travel to every continent at least once.",
    "Learning to play the guitar has been a rewarding journey.",
    "The chef's special is known for its exquisite flavors.",
    "I love capturing moments through photography.",
    "The morning jog around the lake is a refreshing start to the day.",
    "Creative collaboration often leads to innovative solutions.",
    "Clean energy solutions are a priority for a sustainable future.",
    "She received a standing ovation for her closing remarks.",
    "Building meaningful relationships enriches life experiences.",
    "The drive along the coastal highway offers stunning views.",
    "Adapting to change is essential in the rapidly evolving tech world.",
    "The sculpture garden is a peaceful oasis in the bustling city.",
    "The coffee shop on the corner makes the perfect cappuccino.",
    "I enjoy exploring historical landmarks and museums.",
    "What hobbies do you pursue in your free time?",
    "He is known for his sharp wit and humor.",
    "The documentary delves deep into space exploration history."
]

In [ ]:
import numpy as np

# Ensure this line is present and runs before using the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to perform inference
def predict_sentiment(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="tf", padding=True, truncation=True, max_length=512)

    # Get model predictions
    logits = model(inputs.input_ids, attention_mask=inputs.attention_mask).logits

    # Convert logits to probabilities
    prediction_probs = tf.nn.softmax(logits, axis=-1).numpy()

    # Determine predicted class and its probability
    predicted_class_idx = np.argmax(prediction_probs, axis=1)[0]
    predicted_class_prob = prediction_probs[0, 1]

    # Map idx to sentiment label
    sentiment_map = {0: "negative", 1: "positive"}
    predicted_sentiment = sentiment_map[predicted_class_idx]

    return predicted_sentiment, predicted_class_prob

# Test the inference function
for sample_text in sample_texts:
    result,prob = predict_sentiment(sample_text)
    print(f"Sentence: {sample_text}")
    print(f"Sentiment Prediction: {result}, probability: {prob}\n")

Sentence: The movie was fantastic!
Sentiment Prediction: positive, probability: 0.9984567165374756

Sentence: I didn't enjoy the film as much as I thought I would.
Sentiment Prediction: negative, probability: 0.0021556918509304523

Sentence: The plot was predictable, but the acting was top-notch.
Sentiment Prediction: negative, probability: 0.4750937521457672

Sentence: I've never seen anything quite like it. Truly unique experience.
Sentiment Prediction: positive, probability: 0.9982020854949951

Sentence: I would recommend this book to everyone.
Sentiment Prediction: positive, probability: 0.9956117272377014

Sentence: The weather today is quite pleasant compared to yesterday.
Sentiment Prediction: positive, probability: 0.9923932552337646

Sentence: He made a compelling argument during the lecture.
Sentiment Prediction: positive, probability: 0.9840811491012573

Sentence: The new restaurant in town exceeded my expectations.
Sentiment Prediction: positive, probability: 0.995880961418